# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: False


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [4]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [5]:
from sklearn.cluster import KMeans

In [6]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [7]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [8]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [9]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [10]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [18]:
predictions = myKMeans.predict(coordinates.to_numpy())
predictions[:10]

array([22, 18, 72, 84, 78, 55, 87, 26, 31, 25], dtype=int32)

In [22]:
def show_cluster(cluster_number, predictions, center):
    cluster_coord = coordinates.to_numpy()[predictions==cluster_number]
    entries = np.shape(cluster_coord)[0]
    print("number of cluster enries:" , entries)
    
    #create map
    cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
    folium.PolyLine([ [centers[cluster_number,0],centers[cluster_number,1]] , [centers[cluster_number,2],centers[cluster_number,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)
    
    for i in range(entries):
        folium.CircleMarker([cluster_coord[i,0], cluster_coord[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
        folium.CircleMarker([cluster_coord[i,2], cluster_coord[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    return cluster_map
show_cluster(50,predictions,centers)
    
    
    
    


    

number of cluster enries: 2331


## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.

In [23]:
def show_var(predictions, k):
    #get coordinates of the target cluster
    #coordinates were all coordinate collumns from the train data
    np_coord = coordinates.to_numpy()
    
    #compute min and max coordiantes for normalization
    min_cor = np.min(np_coord, axis=0)
    max_cor = np.max(np_coord, axis=0)
    dist = max_cor - min_cor
    print ("max distances:", dist)
    for i in range(k):
        cluster_coord = np_coord[predictions==i]
    
        #compute variances
        var_all = np.var(cluster_coord, axis=0)
        print(i, var_all)

In [24]:
show_var(predictions,100)

max distances: [1.073698 1.177225 0.970017 1.169471]
0 [4.81680687e-05 5.10563868e-05 4.53937232e-05 4.35430068e-05]
1 [2.86131873e-05 2.82136189e-05 2.78889681e-05 2.52838229e-05]
2 [0.00012759 0.00017339 0.00034831 0.0006745 ]
3 [4.57428262e-05 5.29205208e-05 7.53574166e-05 7.99359063e-05]
4 [2.32611658e-04 1.84570830e-04 5.99546497e-05 9.73341138e-05]
5 [9.68515389e-05 7.90329567e-04 1.18990733e-06 4.81022113e-04]
6 [0.00038796 0.00026296 0.00053442 0.00026018]
7 [2.79503101e-05 2.64658389e-05 3.02446425e-05 2.73391986e-05]
8 [5.29560556e-05 4.79430881e-05 3.99124922e-05 4.52705080e-05]
9 [0.00465813 0.00641486 0.00338855 0.00848453]
10 [4.53582842e-05 1.23108850e-04 1.10087022e-03 7.58669694e-04]
11 [0.00053156 0.00033363 0.00066512 0.00045403]
12 [0.00016377 0.00012808 0.00018078 0.00012322]
13 [0.00010459 0.00014029 0.00019883 0.00012831]
14 [3.04045691e-05 2.31626241e-05 3.26955858e-05 5.09784312e-05]
15 [0.00293911 0.00454012 0.00098063 0.00242336]
16 [2.13666130e-04 9.69651586